In [47]:
import numpy as np
import pandas as pd
import math as mt
from sklearn.model_selection import train_test_split


def read_file_basic_preprocess():
    train_df = pd.read_csv("train.csv", index_col='PassengerId')
    test_df = pd.read_csv("test.csv", index_col='PassengerId')
    train_df.drop(['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
    test_df.drop(['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
#     idx = train_df[train_df['Age'].isnull()].index.tolist()
#     str1 = train_df['Age'].astype(str)
#     print(str1[~((str1.str.split('.', n=1).str[1].isnull()) | (str1.str.split('.', n=1).str[1]=='0'))])
    train_age_col_mean = np.mean(train_df['Age'])
    train_df['Age'] = train_df['Age'].fillna(train_age_col_mean).round(2)
#     print(train_df.loc[idx, 'Age'])
#     idx = train_df[train_df['Embarked'].isnull()].index.tolist()
    train_embarked_col_most_freq = train_df['Embarked'].value_counts().index[0]
    train_df['Embarked'].fillna(train_embarked_col_most_freq, inplace=True)
    
    train_fare_col_mean = np.mean(train_df['Fare'])
    test_df['Age'] = test_df['Age'].fillna(train_age_col_mean).round(2)
    test_df['Embarked'].fillna(train_embarked_col_most_freq, inplace=True)
    test_df['Fare'] = test_df['Fare'].fillna(train_fare_col_mean).round(2)
    
    X = train_df.drop('Survived', axis=1)
    y = train_df['Survived']
    return X, y, test_df
    
# X, y = read_file_basic_preprocess()    

In [12]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22.0,1,0,7.2500,S
2,1,female,38.0,1,0,71.2833,C
3,3,female,26.0,0,0,7.9250,S
4,1,female,35.0,1,0,53.1000,S
5,3,male,35.0,0,0,8.0500,S


In [48]:
def handle_categorical_data(X, test_df):
    X_no_recs = len(X.index)
    X_temp = pd.concat([X, test_df])
    X_temp = pd.get_dummies(X_temp)
    X = X_temp.iloc[:X_no_recs,:]
    test_df = X_temp.iloc[X_no_recs:,:]
    return X, test_df

In [14]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
1,3,22.0,1,0,7.2500,0,1,0,0,1
2,1,38.0,1,0,71.2833,1,0,1,0,0
3,3,26.0,0,0,7.9250,1,0,0,0,1
4,1,35.0,1,0,53.1000,1,0,0,0,1
5,3,35.0,0,0,8.0500,0,1,0,0,1


In [50]:
if __name__ == '__main__':
    X, y, test_df = read_file_basic_preprocess()
    X, test_df = handle_categorical_data(X, test_df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    X_train.to_csv('X_train.csv')
    X_test.to_csv('X_test.csv')
    y_train.to_csv('y_train.csv')
    y_test.to_csv('y_test.csv')
    test_df.to_csv('Unseen_test.csv')

In [51]:
X_train.nunique()

Pclass          3
Age            86
SibSp           7
Parch           7
Fare          222
Sex_female      2
Sex_male        2
Embarked_C      2
Embarked_Q      2
Embarked_S      2
dtype: int64

In [24]:
len(y_train) - y_train[y_train == 1].sum() 

410

In [29]:
(y_train == 1).sum()

258

In [30]:
258/ 668

0.38622754491017963

In [108]:
multi_pass_tick = train_df['Ticket'].value_counts()>1
multi_pass_tick[multi_pass_ticket==True].index

Index(['CA. 2343', '347082', '1601', 'CA 2144', '347088', '3101295',
       'S.O.C. 14879', '382652', '4133', 'PC 17757',
       ...
       '2659', 'PC 17611', '2908', '13507', '2668', '250644', '11751',
       'PC 17604', '26360', 'WE/P 5735'],
      dtype='object', length=134)

In [90]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,female,38.0,1,0,PC 17599,71.2833,C
3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,female,35.0,1,0,113803,53.1000,S
5,0,3,male,35.0,0,0,373450,8.0500,S


In [91]:
train_df.nunique()

Survived      2
Pclass        3
Sex           2
Age          89
SibSp         7
Parch         7
Ticket      681
Fare        248
Embarked      3
dtype: int64

In [9]:
X.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object